HuggingFace transformers와 Tensorflow를 통해 사전학습모델을 Fine-tuning하는 <br> 방법 및 학습된 모델을 통해 **Multi-class text classfication** 문제를 해결해보자

# Load Dataset
**Load Trainset**

In [1]:
import re
import os
import json
import pandas as pd
import numpy as np

- 데이터 셋의 경우 KLUE에서 Topic-Classification을 위해 사용한 YNAT 데이터 셋을 그대로 사용하였다.<br>
- YNAT는 연합뉴스의 2016-202년까지의 뉴스 headline을 수집한 데이터 셋이며, 총 7가지 클래스(IT과학, 경제, 사회, 생활문화, 세계, 스포츠, 정치)로 분류되어있다.

**Dataset : https://github.com/KLUE-benchmark/KLUE**

In [2]:
# Load Train-set
with open('ynat-v1.1_train.json', mode='rt', encoding='utf-8-sig') as f:
    train_dataset = json.load(f)

train_dataset_list = [{'text':data['title'], 'label':data['label']} for data in train_dataset]
train_df = pd.DataFrame(train_dataset_list)
train_df.head()

,text,label
0,유튜브 내달 2일까지 크리에이터 지원 공간 운영,생활문화
1,어버이날 맑다가 흐려져…남부지방 옅은 황사,생활문화
2,내년부터 국가RD 평가 때 논문건수는 반영 않는다,사회
3,김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것,사회
4,회색인간 작가 김동식 양심고백 등 새 소설집 2권 출간,생활문화


**라벨 별 개수 확인**

In [3]:
# count by label
train_df.groupby(by=['label']).count()

,text
label,
IT과학,5235
경제,6118
사회,5133
생활문화,5751
세계,8320
스포츠,7742
정치,7379


**라벨 인코딩**
- 학습 시 Loss 계산을 하기위해 숫자 형태로 인코딩

In [4]:
# Label Encoding
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(train_df['label'])
num_labels = len(label_encoder.classes_)

train_df['encoded_label'] = np.asarray(label_encoder.transform(train_df['label']), dtype=np.int32)
train_df.head()

,text,label,encoded_label
0,유튜브 내달 2일까지 크리에이터 지원 공간 운영,생활문화,3
1,어버이날 맑다가 흐려져…남부지방 옅은 황사,생활문화,3
2,내년부터 국가RD 평가 때 논문건수는 반영 않는다,사회,2
3,김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것,사회,2
4,회색인간 작가 김동식 양심고백 등 새 소설집 2권 출간,생활문화,3


**모델 검증을 위해 validation set을 training set의 20% 비율로 분리**

In [5]:
train_texts = train_df["text"].to_list() # Features (not-tokenized yet)
train_labels = train_df["encoded_label"].to_list() # Labels

In [6]:
from sklearn.model_selection import train_test_split

# Split Train and Validation data
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.2, random_state=0)

# Tokenizing the text

본격적으로 Tokenizing 및 pretrained 모델 사용을 위해 🤗HuggingFace의 Transformers 라이브러리를 활용한다. <br>
Transformers를 통해 저장된 모델은 기본적으로 pretrained model, tokenizer, vocab, config 파일 등을 포함하고 있으며, **from_pretrained()** 메소드를 통해 로드할 수 있다.

KLUE-BERT Model Path

- **K**orean **L**anguage **U**nderstanding **E**valuation

In [7]:
HUGGINGFACE_MODEL_PATH = "klue/bert-base"

여기서 이용할 KLUE-BERT모델 또한 HuggingFace Model Hub에 배포되어 있으며 해당 모델 주소를 추후 **from_pretrained()**에 인자로 넣어주어 모델을 다운로드 및 로드할 수 있음

**Tokenizer 로드 및 Tokenizing**



In [8]:
from transformers import BertTokenizerFast

# Load Tokenizer
tokenizer = BertTokenizerFast.from_pretrained(HUGGINGFACE_MODEL_PATH)

# Tokenizing
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

**참고** <br>
- Tokenizer는 BertTokenizer(), BertTokenizerFast() 무엇을 사용하던 상관없지만, BertTokenizerFast()가 BertTokenizer() 대비 1.2 ~ 1.5배 tokenizing 속도가 빠르다. (단, BertTokenizerFast()는 성능에 영향을 줄 수 있으니 유의하자)
<br><br>

- truncation혹은 padding 옵션을 주어 input sequence의 길이를 맞춰줄 수 있으며, 여러가지 옵션으로 세세한 튜닝도 가능하다.

# Creating a Dataset object for Tensorflow
fine-tuning을 진행하기 전에 먼저 tokenized 된 데이터 셋을 Tensorflow의 Dataset object로 변환을 위해 from_tensor_slices()메서드를 수행한다.

In [9]:
import tensorflow as tf

# trainset-set
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))

# validation-set
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))

# Fine-tuning BERT
Fine-tuning을 위해 tensorflow를 이용

**Load Pretrained Model**

In [10]:
#tensorflow 버전이 안맞으면 import가 안될 수 있음
#!pip install --user tensorflow==2.5.0

In [11]:
import tensorflow as tf

print(tf.__version__)

2.5.0


In [12]:
from transformers import TFBertForSequenceClassification

num_labels = len(label_encoder.classes_) # .class_ 메소드로 라벨 개수를 얻음
model = TFBertForSequenceClassification.from_pretrained(HUGGINGFACE_MODEL_PATH, num_labels=num_labels, from_pt=True) # from_pt=True를 넣어서 텐서모델로 변환 및 로드가능

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**Training**

In [13]:
from tensorflow.keras.callbacks import EarlyStopping

callback_earlystop = EarlyStopping(
    monitor="val_accuracy", 
    min_delta=0.001, # the threshold that triggers the termination (acc should at least improve 0.001)
    patience=2)

model.fit(
    train_dataset.shuffle(1000).batch(16), epochs=5, batch_size=16,
    validation_data=val_dataset.shuffle(1000).batch(16),
    callbacks = [callback_earlystop]
)

Epoch 1/5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: '<' not supported between instances of 'Literal' and 'str'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: '<' not supported between instances of 'Literal' and 'str'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


C:\Users\ebdl\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\autograph\impl\api.py:376: FutureWarning: The old compute_loss method is deprecated as it conflicts with the Keras compute_loss method added in TF 2.8. If you want the original HF compute_loss, please call hf_compute_loss() instead. From TF versions >= 2.8, or Transformers versions >= 5, calling compute_loss() will get the Keras method instead.
  return py_builtins.overload_of(f)(*args)


2284/2284 [==============================] - 223s 94ms/step - loss: 0.4313 - accuracy: 0.8561 - val_loss: 0.3483 - val_accuracy: 0.8771
Epoch 2/5
2284/2284 [==============================] - 212s 93ms/step - loss: 0.2785 - accuracy: 0.9044 - val_loss: 0.3800 - val_accuracy: 0.8748
Epoch 3/5
2284/2284 [==============================] - 214s 94ms/step - loss: 0.1905 - accuracy: 0.9343 - val_loss: 0.4564 - val_accuracy: 0.8723


training 방법은 tf.keras에서 모델을 훈련할 때와 같음. training 사전 종료를 위해 EarlyStopping callback 함수를 적용

# Saving Model

transformers에서 제공하는 save_pretrained() 메소드를 사용하면 모델을 저장할 수 있다.<br>

모델을 저장하게 되면 총 5가지의 파일이 저장 위치에 생성되며, 추후 해당 파일을 그대로 HuggingFace Model Hub로 포팅하여 손쉽게 로드할 수 있다.

In [14]:
# Change id2label, label2id in model.config

id2labels = model.config.id2label
model.config.id2label = {id : label_encoder.inverse_transform([int(re.sub('LABEL_', '', label))])[0]  for id, label in id2labels.items()}

label2ids = model.config.label2id
model.config.label2id = {label_encoder.inverse_transform([int(re.sub('LABEL_', '', label))])[0] : id   for id, label in id2labels.items()}

학습된 모델은 기본적으로 모델 아키텍처, 레이어, label과 같은 모델 정보를 **config** 속성에 저장하게 된다. <br>
이 **config**에는 ***id2label, label2id*** 라는 ***index값과 label 속성***이 매핑된 정보가 존재하는데, <br>
우리가 위에서 LabelEncoder를 통해 label을 숫자형태로 encoding을 하여 학습하였기 때문에 해당 속성들 또한 encoding된 형태로 저장되어 있다.<br><br>
그래서 이를 다시 **decoding함으로써 본래의 label 값을 갖도록 변환**한다.<br>

In [15]:
# Saving the model and tokenizer

MODEL_NAME = 'fine-tuned-klue-bert-base'
MODEL_SAVE_PATH = os.path.join("_model", MODEL_NAME) # change this to your preferred location

if os.path.exists(MODEL_SAVE_PATH):
    print(f"{MODEL_SAVE_PATH} -- Folder already exists \n")
else:
    os.makedirs(MODEL_SAVE_PATH, exist_ok=True)
    print(f"{MODEL_SAVE_PATH} -- Folder create complete \n")

# save tokenizer, model
model.save_pretrained(MODEL_SAVE_PATH)
tokenizer.save_pretrained(MODEL_SAVE_PATH)

_model\fine-tuned-klue-bert-base -- Folder already exists 



('_model\\fine-tuned-klue-bert-base\\tokenizer_config.json',
 '_model\\fine-tuned-klue-bert-base\\special_tokens_map.json',
 '_model\\fine-tuned-klue-bert-base\\vocab.txt',
 '_model\\fine-tuned-klue-bert-base\\added_tokens.json',
 '_model\\fine-tuned-klue-bert-base\\tokenizer.json')

**save_pretrained() 메소드를 통해 model,tokenizer를 저장**

경로를 Hugging space의 개인 경로로 하게 되면 온라인에서 사전에 학습한 모델을 불러올 수 있다! <br>
(나는 로컬에 저장하였다)

# Load the saved model and prediction

**Loading the model and tokenizer**

In [16]:
from transformers import TextClassificationPipeline

# Load Fine-tuning model
loaded_tokenizer = BertTokenizerFast.from_pretrained(MODEL_SAVE_PATH)
loaded_model = TFBertForSequenceClassification.from_pretrained(MODEL_SAVE_PATH)

text_classifier = TextClassificationPipeline(
    tokenizer=loaded_tokenizer, 
    model=loaded_model, 
    framework='tf',
    return_all_scores=True
)

Some layers from the model checkpoint at _model\fine-tuned-klue-bert-base were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at _model\fine-tuned-klue-bert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.
C:\Users\ebdl\Anaconda3\lib\site-packages\tra

**Load Testset**

In [17]:
import string
import re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

def clean_text(text):
    cleaned_text = text.lower().strip() #  # 텍스트를 소문자로 변환하고 양쪽의 공백을 제거
    # cleaned_text = re.sub(r'\d+', '', cleaned_text) # 텍스트에서 숫자를 제거
    cleaned_text = "".join(char for char in cleaned_text if char not in string.punctuation) # 문장부호를 제거
    stop_words = set(stopwords.words('english'))  # 영어 불용어 집합을 가져옴
    cleaned_text = " ".join(word for word in cleaned_text.split() if word not in stop_words) # 불용어를 제거
    return cleaned_text


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ebdl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
# Load Test-set
with open('ynat-v1.1_dev.json', mode='rt', encoding='utf-8-sig') as f:
    test_dataset = json.load(f)

test_dataset_list = [{'text':clean_text(data['title']), 'label':data['label']} for data in test_dataset]
test_df = pd.DataFrame(test_dataset_list)
test_df.head()

,text,label
0,5억원 무이자 융자는 되고 7천만원 이사비는 안된다,사회
1,왜 수소충전소만 더 멀리 떨어져야 하나 한경연 규제개혁 건의,사회
2,항응고제 성분 코로나19에 효과…세포실험서 확인,IT과학
3,실거래가 가장 비싼 역세권은 신반포역…33㎡당 1억 육박,경제
4,기자회견 하는 성 소수자 단체,사회


**Prediction using Pipelines**

In [19]:
predicted_label_list = []
predicted_score_list = []

for text in test_df['text']:
    # predict
    preds_list = text_classifier(text)[0]

    sorted_preds_list = sorted(preds_list, key=lambda x: x['score'], reverse=True)
    predicted_label_list.append(sorted_preds_list[0]['label']) # label
    predicted_score_list.append(sorted_preds_list[1]['score']) # score

In [20]:
test_df['pred'] = predicted_label_list
test_df['score'] = predicted_score_list
test_df.head()

,text,label,pred,score
0,5억원 무이자 융자는 되고 7천만원 이사비는 안된다,사회,경제,0.004280
1,왜 수소충전소만 더 멀리 떨어져야 하나 한경연 규제개혁 건의,사회,사회,0.038732
2,항응고제 성분 코로나19에 효과…세포실험서 확인,IT과학,IT과학,0.002602
3,실거래가 가장 비싼 역세권은 신반포역…33㎡당 1억 육박,경제,경제,0.003396
4,기자회견 하는 성 소수자 단체,사회,사회,0.004397


pred만 보면됨

# Evaluation

In [21]:
from sklearn.metrics import classification_report

In [22]:
test_df['label']

0         사회
1         사회
2       IT과학
3         경제
4         사회
        ... 
9102      경제
9103      사회
9104      경제
9105      사회
9106      사회
Name: label, Length: 9107, dtype: object

In [23]:
#test_df['pred'] = test_df['pred'].apply(lambda x: x['label'])

In [24]:
#test_df['pred']

In [25]:
print(classification_report(y_true=test_df['label'], y_pred=test_df['pred']))

              precision    recall  f1-score   support

        IT과학       0.68      0.83      0.75       554
          경제       0.83      0.81      0.82      1348
          사회       0.88      0.82      0.85      3701
        생활문화       0.79      0.89      0.84      1369
          세계       0.89      0.83      0.86       835
         스포츠       0.93      0.92      0.93       578
          정치       0.80      0.86      0.83       722

    accuracy                           0.84      9107
   macro avg       0.83      0.85      0.84      9107
weighted avg       0.84      0.84      0.84      9107



scikit-learn의 classification_report를 통해 label별 결과를 확인한다.<br>
f1-score가 0.84로 기존 KLUE-BERT-BASE의 Topic Classification 점수인 85.49와 비슷한 오차범위 내로 성능 재현이 이루어졌다고 할 수 있을 것 같다.